In [1]:
!pip install transformers
!pip install rank_bm25
!pip install konlpy

In [2]:
import sklearn
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import pandas as pd
import rank_bm25
from rank_bm25 import BM25Okapi
from sklearn.preprocessing import StandardScaler
import konlpy
from konlpy.tag import Okt, Hannanum, Kkma

In [3]:
#news = pd.read_csv('news.csv')
news = pd.read_csv('samsung_feb.csv')
print(news.head())

#news = news[:1000]
print(len(news))
news_contexts = news['contents']
print(f'{len(news_contexts)=}')

  section                                       title date  page  \
0     NaN  1월 수출 18%↑·8개월째 무역흑자…반도체·中 수출 동시 '플러스'(3보)  NaN   NaN   
1     NaN            삼성 VXT, 英으로부터 국제 정보보호 관리체계 표준 인정  NaN   NaN   
2     NaN                삼성 VXT, 국제 정보보호 관리체계 표준 인정받아  NaN   NaN   
3     NaN                  삼성 VXT, 국제 정보보호 관리체계 표준 인정  NaN   NaN   
4     NaN     MSCI, 삼성전자 ESG 등급 'AA'로 상향…"지배구조 관행 개선"  NaN   NaN   

                                                 url    relate_stock  \
0  https://n.news.naver.com/mnews/article/003/001...  ['삼성전자', '기아']   
1  https://n.news.naver.com/mnews/article/277/000...        ['삼성전자']   
2  https://n.news.naver.com/mnews/article/417/000...        ['삼성전자']   
3  https://n.news.naver.com/mnews/article/018/000...        ['삼성전자']   
4  https://n.news.naver.com/mnews/article/421/000...        ['삼성전자']   

                                   real_title  \
0  1월 수출 18%↑·8개월째 무역흑자…반도체·中 수출 동시 '플러스'(3보)   
1            삼성 VXT, 英으로부터 국제 정보보호 관리체계 표준 인정   
2          

In [4]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

model_name = "ainize/kobart-news"

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
okt=Okt()
model = BartForConditionalGeneration.from_pretrained(model_name)

cnt=0
idx=[]
for i in range(len(news_contexts)):
  #tokens=tokenizer.tokenize(news_contexts[i])
  tokens=okt.morphs(news_contexts[i])
  if len(tokens)>512:
    cnt+=1
    idx.append(i)

print(cnt)
print(idx)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of 

1695
[0, 6, 8, 13, 17, 20, 21, 22, 25, 28, 29, 30, 31, 33, 35, 36, 37, 39, 40, 41, 54, 56, 58, 61, 62, 63, 66, 68, 71, 72, 76, 81, 82, 83, 94, 95, 99, 100, 101, 104, 107, 109, 110, 111, 112, 113, 114, 115, 116, 119, 121, 123, 124, 125, 129, 130, 131, 132, 133, 134, 135, 136, 137, 141, 143, 145, 146, 148, 152, 153, 154, 155, 156, 157, 158, 159, 160, 163, 170, 173, 175, 177, 182, 185, 191, 193, 197, 199, 202, 203, 204, 206, 208, 209, 212, 216, 219, 220, 221, 223, 224, 225, 230, 231, 232, 234, 235, 237, 240, 242, 243, 244, 246, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 267, 268, 269, 271, 273, 275, 280, 282, 284, 285, 290, 292, 294, 295, 296, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 313, 314, 315, 318, 319, 320, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 355, 356, 357, 358, 359, 360, 362, 363, 364,

In [144]:
import re

news_contexts = news['contents']
print(news_contexts[956])

def remove_space(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'^\s+', '', text)
    text = re.sub(r'\s+$', '', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\r', ' ', text)
    return text

# 따옴표 제거
def remove_quote(text):
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\`', ' ', text)
    text = re.sub(r'\‘', ' ', text)
    text = re.sub(r'\’', ' ', text)

    text = re.sub(r'\"', ' ', text)
    text = re.sub(r'\“', ' ', text)
    text = re.sub(r'\”', ' ', text)
    return text

# () 괄호 안 제거
def remove_between_round_brackets(text):
    return re.sub(r'\([^)]*\)', ' ', text)

# {} 괄호 안 제거
def remove_between_curly_brackets(text):
    return re.sub(r'\{[^}]*\}', ' ', text)

# [] 괄호 안 제거
def remove_between_square_brackets(text):
    return re.sub(r'\[[^]]*\]', ' ', text)

# <> 괄호 안 제거
def remove_between_angle_brackets(text):
    return re.sub(r'\<[^>]*\>', ' ', text)

# url 제거
def remove_url(text):
    text = re.sub(r'http[s]?://(?:[\t\n\r\f\v]|[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)
    text = re.sub(r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', ' ', text)
    return text

# 이메일 제거
def remove_email(text):
    return re.sub(r'[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',' ', text)

# 날짜 제거
def remove_date(text):
    return re.sub(r'\d+[.]\d+[.]\d+',' ', text)

# 숫자 이용 문자 제거
def remove_number_text(text):
    text = re.sub(r'[0-9]+%', ' ', text)
    text = re.sub(r'[0-9]+층', ' ', text)
    return text

# 특수문자 제거
def remove_symbol(text):
    return re.sub(r'[^\w\s.,]', ' ', text)

# 자음, 모음 제거
def remove_consonants_vowels(text):
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', text)
    return text

# 앞에 날리기
def remove_to_callback(news):
    callback_list = []
    try :
        for b in re.finditer('flash_removeCallback()',news):
            callback_list.append(b.end())
        news = news[callback_list[0]:]
    except :
        news = news
    return news

# 뒤에 날리기
def remove_from_back_third_triangle(news):
    triangle_list = []
    for a in re.finditer('▶',news):
        triangle_list.append(a.start())
    if len(triangle_list) >= 3:
        news = news[:triangle_list[-3]]
    elif len(triangle_list) == 2:
        news = news[:triangle_list[-2]]
    elif len(triangle_list) == 1:
        news = news[:triangle_list[-1]]
    return news

def remove_float(news):
    return re.sub(r'[0-9]+.[0-9]*',' ', news)

def remove_journalist(news):
    return re.sub(r'[가-핳]{2,4}\s+기자[^가-핳]+',' ', news)


def pre_processing(txt):
    txt = remove_to_callback(txt)
    txt = remove_from_back_third_triangle(txt)
    txt = remove_email(txt)
    #txt = remove_date(txt)
    txt = remove_quote(txt)
    txt = remove_url(txt)
    txt = remove_number_text(txt)
    txt = remove_consonants_vowels(txt)
    txt = remove_between_round_brackets(txt)
    txt = remove_between_curly_brackets(txt)
    txt = remove_between_square_brackets(txt)
    txt = remove_between_angle_brackets(txt)
    txt = remove_journalist(txt)
    txt = remove_float(txt)
    txt = remove_symbol(txt)
    txt = remove_space(txt)
    return txt

news_contexts = list(map(pre_processing, news_contexts))


(서울=연합뉴스) 류영석 기자 = 이재용 삼성전자 회장이 6일 오후 출국을 위해 서울 강서구 서울김포비즈니스항공센터에 들어서고 있다. 2024.2.6 ondol@yna.co.kr


In [145]:
print(news_contexts[956])
#print(len(tokenizer.tokenize(news_contexts[6])))
print(len(okt.morphs(news_contexts[956])))

이재용 삼성전자 회장이 오후 출국을 위해 서울 강서구 서울김포비즈니스항공센터에 들어서고 있다. .
22


In [146]:
# print(news_contexts[6])
# print(tokenizer.tokenize(news_contexts[6]))
# print(len(tokenizer.tokenize(news_contexts[6])))
# print(len(tokenizer.encode(news_contexts[6])))
# print(tokenizer.decode(tokenizer.encode(news_contexts[6])))

print(news_contexts[6])
print(okt.morphs(news_contexts[6]))
print(len(okt.morphs(news_contexts[6])))
print(' '.join(okt.morphs(news_contexts[6])))


산업부, 반도체 수출 증가 반도체 수요 회복에 대중 수출 반도체 장비 수출 통제 가능성은 부담홍해 사태로 운임 등 물류기업 부담 증가 부산항에 컨테이너가 쌓여 있다. 연합뉴스.반도체 경기 회복에 힘입어 대중국 수출이 월 만에 반등했다. 자동차까지 최대 실적을 올리며 수출 증가세가 월 연속 이어지고 있다. 그러나 홍해 사태에 따른 물류 차질에다, 반도체를 둘러싼 미국 중국 갈등이 새로운 국면에 접어들면서 수출 전망은 안갯속이다.산업통상자원부는 지난달 수출액이 만 달러로 전년 동기보다 증가했다고 밝혔다. 동안 부진한 모습을 보였던 수출은 지난해 증가세로 반등한 이후, 월 연속 증가세를 이어갔다. 기준 역대 최대 수출액을 기록했던 에는 미치지 못하지만 수출 회복세가 본격화되는 양상이다.수출 증가세는 반도체가 주도했다. 지난달 반도체 수출액은 만 달러로 전년 동월 대비 늘었다. 고대역폭메모리 등 고부가가치 메모리 판매 확대와 수급 개선으로 가격 상승세가 이어지면서 반도체 실적이 개선됐다.같은 기간 자동차 수출도 늘며 역대 기준 최대 실적인 만 달러를 달성했다. 수출 단가가 높은 친환경차와 대형 스포츠유틸리티차 수출이 호조세를 이어갔다고 산업부는 설명했다.일반 기계 , 가전 , 디스플레이 , 선박 , 석유화학 , 바이오헬스 등 품목 수출도 전보다 늘었다.지난달 대중 반도체 수출 증가지난달 대중 수출액은 만 달러로, 전년 대비 늘어나며 미국을 제치고 다시 최대 수출국이 됐다. 대중 수출이 증가한 것은 월 만이다. 특히 최대 수출품목인 반도체 수출이 지난해 같은 기간보다 늘어난 영향이 컸다.조익노 산업부 무역정책관은 중국 수출의 가 반도체, 디스플레이, 휴대폰 등 정보기술 품목 이라며 통상 비수기로 보는 기를 거쳐 분기로 갈수록 업황이 회복되면서 대중 수출도 확대될 것으로 본다 고 말했다.또한 대미 수출액은 증가한 만 달러를 기록했다. 대미 수출에서 자동차 , 기계 품목도 큰 폭으로 늘었다.산업부 관계자는 미국 경기 호황으로 소비 심리가 좋은 데다, 신규 투자 수요가 늘며

In [147]:
for i in range(10):
  print(news_contexts[i])

산업부, 수출입동향 수출 억弗 수입 억弗中 월만에 반등 반도체 월만에 최대 증가율 전진환 기자 인천 연수구 인천신항 컨테이너 터미널에 컨테이너가 쌓여있다. 임소현 지난달 수출이 전년 대비 증가하며 두자릿수 플러스를 달성한 가운데 무역수지도 월 연속 흑자행진을 이어가고 있다. 대 중국 수출이 월 만에 플러스로 전환했고, 반도체 수출은 증가해 월 만에 최대 증가율을 다시 썼다. 산업통상자원부가 발표한 수출입 동향 에 따르면 수출은 전보다 증가한 만 달러 를 기록했다.지난해 수출이 월 만에 증가로 돌아선 이후 월 연속 플러스를 지속한 것이다.지난달 수입은 줄어 만 달러 를 나타냈다.이에 우리나라 무역수지는 달러 흑자였다. 무역수지는 월 연속 흑자로 집계됐다.지난달 수출은 우리 최대 수출품목인 반도체 수출이 월만에 최대 증가율인 를 기록한 데다 우리나라 최대 수출국인 대중 수출이 월만에 플러스로 전환된 것이 증가세를 이끌었다.특히 주력 수출품목 중 품목 수출이 증가하며 이후 최대 플러스 품목 수를 기록했다. 반도체 수출은 전년 대비 증가한 이후 월 만에 최대 증가율인 를 기록하면서 월 연속 플러스흐름을 이어갔다. 삼성전자 평택 캠퍼스 반도체 공장 내부. 재판매 및 DB 금지자동차 수출 도 역대 기준 최대 실적인 달러 를 기록하며 월 연속 증가했다. 또한 일반기계 는 월, 가전 은 월, 디스플레이 선박 은 월, 석유화학 바이오헬스 는 월 연속 수출 증가세를 기록했다. 한편 컴퓨터 수출은 이후 월 동안 이어진 마이너스 고리를 끊었으며 철강 , 석유제품 , 자동차부품 , 섬유 수출도 플러스로 전환됐다.또한 주요 수출시장 중 독립국가연합 을 제외한 시장에서 수출이 증가했다. 우리나라 최대 수출국인 대중 수출 은 달러 로 이후 월 만에 플러스로 전환됐다.대미국 수출 도 월 연속 증가흐름을 이어가는 가운데 역대 중 최대 수출실적인 달러 를 기록했다. 특히 중국과 미국은 각각 월, 월 연속 달러 이상 수출실적을 이어나갔다. 한편 역대 중 최대 수출실적 을 기록한 인도 를 포함, 

In [148]:
#print(len(tokenizer.tokenize(news_contexts[6])))
print(len(okt.morphs(news_contexts[6])))


798


#TF-IDF

In [149]:
#tfidf_vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, min_df = 3, ngram_range=(1,3))
tfidf_vectorizer = TfidfVectorizer(tokenizer = okt.morphs, min_df = 3, ngram_range=(1,2))
tfidf_vectorizer.fit(news_contexts)
context_vector = tfidf_vectorizer.transform(news_contexts).toarray()

# scaler = StandardScaler()
# context_vector = scaler.fit_transform(context_vector)
# print(tfidf_vectorizer.vocabulary_)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [154]:
print(len(context_vector[8]))

91715


#BM-25

In [52]:
# tokenized_corpus = [okt.morphs(c) for c in news_contexts]
# print((tokenized_corpus[9:11]))
# bm25 = BM25Okapi(tokenized_corpus)

[['부터', '연속', 'A', '등급', '작년', '사회', '지배구조', '평가', '상승', '해', 'AA', '로', '황진환', '기자', '삼', '성', '전자', '는', '글로벌', 'ESG', '평가', '기관', '인', 'MSCI', '에서', '발표', '한', 'MSCI', 'ESG', '평가', '에서', 'AA', '등급', '을', '받았다고', '밝혔다', '.', 'AA', '등급', '은', '국내', 'ICT', '업계', '최고', '수준', '이다', '.', 'MSCI', 'ESG', '평가', '는', '글로벌', '투자', '은행', '인', '모건스탠리', '의', '자회사', '인', 'MSCI', '가', '부터', '실시', '하는', '글로벌', 'ESG', '평가', '다', '.', 'MSCI', '는', '매년', '전', '세계', '개', '상장', '사를', '대상', '으로', 'ESG', '경영', '현황', '을', '평가', '해', '등급', '으로', '분류', '한다', '.', '삼', '성', '전자', '는', '부터', '연속', 'A', '등급', '을', '받았고', ',', '지난해', '사회', '와', '지배구조', '부문', '의', '평가', '점수', '가', '비교', '적', '큰', '폭', '으로', '오르며', '전체', '등급', '이', '한', '단계', '올랐다', '.', '구체', '적', '으로', '글로벌', '인재', '양성', '자원', '조달', '정책', '기업', '지배구조', '등', '부문', '에서', '점수', '가', '올랐다', '.', '삼', '성', '전자', '는', '대', '내외', '불확실한', '경', '영', '환경', '에서', '지속', '추진', '한', '글로벌', '지속', '가능', '경영', '활동', '이', '인정받은', '것', '이라고', '설명', '했다', '.', '삼', 

In [64]:
# print(len(bm25.get_scores(tokenized_corpus[0])))
# print(len(bm25.get_scores(tokenized_corpus[1])))
# print(len(bm25.get_scores(tokenized_corpus[2])))

3288
3288
3288


In [65]:
#context_vector = [bm25.get_scores(c) for c in tokenized_corpus]

In [153]:
scaler = StandardScaler()
context_vector = scaler.fit_transform(context_vector)
print(tfidf_vectorizer.vocabulary_)

{'산업부': 40360, ',': 0, '수출입': 44459, '동향': 24104, '수출': 44424, '억': 48632, '弗': 5256, '수입': 44331, '월': 56059, '만에': 30501, '반등': 34705, '반도체': 34460, '최대': 78898, '증가': 75109, '율': 57043, '기자': 15523, '인천': 67208, '연수구': 52820, '신': 45918, '항': 87967, '컨테이너': 80442, '터미널': 81465, '에': 49195, '가': 5305, '쌓여있다': 46658, '.': 1393, '임소현': 67932, '지난달': 75677, '이': 62844, '전': 70579, '년': 18265, '대비': 21767, '하며': 85851, '두': 24861, '자릿수': 68985, '플러스': 84517, '를': 28431, '달성': 21133, '한': 86474, '가운데': 6892, '무역수지': 33058, '도': 22908, '연속': 52776, '흑자': 91542, '행진': 89174, '을': 59146, '어가': 48441, '고': 10392, '있다': 68445, '대': 21399, '중국': 74718, '로': 27411, '전환': 71642, '했고': 88907, '은': 58117, '해': 88045, '다시': 20553, '썼다': 46682, '산업': 40293, '통상': 81797, '자원부': 69178, '발표': 35251, '따르면': 26220, '전보': 71097, '다': 20292, '만': 30251, '달러': 21052, '기록': 15001, '했다': 88939, '지난해': 75748, '돌아선': 23815, '이후': 66171, '지속': 76168, '것': 9203, '이다': 64718, '줄어': 74427, '나타냈다': 17036, '우리나라': 554

In [155]:
print(type(context_vector))
print(len(context_vector))
print(len(context_vector[0]), len(context_vector[100]), len(context_vector[1000]))
print(context_vector[2])

<class 'numpy.ndarray'>
3288
91715 91715 91715
[ 0.3630159  -0.10297005 -0.02885533 ... -0.05071252 -0.02883368
 -0.03488366]


In [160]:
model = DBSCAN(eps=0.6, min_samples=3, metric = "cosine")
result = model.fit_predict(context_vector)
train_extract={}
train_extract['cluster1st'] = result

print('군집개수 :', result.max())
train_extract

군집개수 : 208


{'cluster1st': array([ -1,   0,   0, ..., 208,  -1,  -1])}

In [161]:
print(result[51])

-1


In [162]:
from collections import Counter
counter = Counter(result)
print(sorted(counter.items(), reverse=True, key = lambda x: x[1]))


[(-1, 1912), (27, 120), (33, 34), (177, 24), (82, 23), (31, 22), (62, 22), (97, 22), (63, 20), (160, 20), (0, 19), (116, 19), (200, 19), (90, 18), (127, 15), (131, 15), (146, 15), (175, 15), (64, 14), (186, 14), (188, 14), (9, 13), (106, 13), (164, 13), (47, 12), (126, 12), (128, 12), (130, 12), (174, 12), (98, 11), (48, 10), (50, 10), (86, 10), (91, 10), (140, 10), (154, 10), (191, 10), (1, 9), (72, 9), (144, 9), (34, 8), (53, 8), (57, 8), (81, 8), (192, 8), (207, 8), (11, 7), (23, 7), (20, 7), (25, 7), (92, 7), (145, 7), (159, 7), (193, 7), (8, 6), (19, 6), (21, 6), (36, 6), (49, 6), (95, 6), (93, 6), (117, 6), (122, 6), (129, 6), (141, 6), (178, 6), (205, 6), (206, 6), (2, 5), (7, 5), (10, 5), (12, 5), (18, 5), (37, 5), (39, 5), (59, 5), (66, 5), (80, 5), (89, 5), (100, 5), (118, 5), (121, 5), (134, 5), (135, 5), (142, 5), (147, 5), (4, 4), (16, 4), (28, 4), (30, 4), (61, 4), (35, 4), (41, 4), (43, 4), (46, 4), (54, 4), (55, 4), (58, 4), (67, 4), (71, 4), (73, 4), (75, 4), (78, 4), 

In [47]:
#for i,_ in sorted(counter.items(), reverse=True, key = lambda x: x[1]):


In [163]:
for i, c in enumerate(result):
  if c==27:

    print('idx:', i,'  -> ', news_contexts[i])
    #print('length :',len(tokenizer.tokenize(news_contexts[i])))
    print('length :',len(okt.morphs(news_contexts[i])))
    print()

idx: 407   ->  한국무역협회가 오늘 이재용 삼성전자 회장이 경영권 승계 관련 부당 합병 회계 부정 혐의에 대해 에서 무죄를 선고받은 것에 대해 환영한다는 뜻을 밝혔습니다.무협은 이번 판결을 계기로 글로벌 기업 삼성의 사법리스크가 해소돼 결과적으로 우리 수출과 경제 활성화에 도움이 될 것으로 기대한다 고 전했습니다.이어 특히 최근 반도체 수요가 회복되고, 첨단산업 투자에 대한 글로벌 경쟁이 치열한 현재의 여건을 감안하면 판결에 대해 다행스럽게 생각한다 고 덧붙였습니다.아울러 무협은 앞으로 삼성이 더욱 진취적인 전략을 통해 인공지능 등 첨단기술을 선도하는 글로벌 기업으로서 국민으로부터 보다 신뢰받기를 바란다 고 강조했습니다.당신의 제보가 뉴스로 만들어집니다.SBS Biz는 여러분의 제보를 기다리고 있습니다.홈페이지
length : 158

idx: 409   ->  이재용 삼성전자 회장이 경영권 승계 관련 부당 합병 회계 부정 혐의에 대한 선고 공판에서 무죄를 선고받자, 경제단체들은 환영한다는 뜻을 밝혔다. 이재용 삼성전자 회장이 오후 서울 서초구 서울중앙지방법원에서 열린 회계부정 부당합병 관련 선고 공판에 출석하고 있다. 이 회장은 이날 무죄를 선고받았다. 뉴스 강석구 대한상공회의소 조사본부장은 삼성물산 제일모직 합병 건과 관련해 이재용 회장에게 무죄를 선고한 판결을 적극 환영한다 며 이번 판결은 첨단산업 주도권을 확보하기 위한 우리 기업들의 글로벌 경쟁과 이제 막 회복세에 들고 있는 우리 경제에 큰 도움이 될 것 이라고 평가했다.김고현 한국무역협회 전무는 이번 판결을 계기로 글로벌 기업 삼성의 사법 리스크가 해소돼 우리 수출과 경제 활성화에 도움이 될 것으로 기대한다 며 최근 반도체 수요가 회복되고 첨단산업 투자에 대한 글로벌 경쟁이 치열한 현재 여건을 감안하면 판결에 대해 다행스럽게 생각한다 고 밝혔다 한국경영자총연합회도 경영계는 법원의 판단을 존중하며, 금번 판결을 통해 지금까지 제기됐던 의혹과 오해들이 해소되어 다행 이라고 밝혔다. 이어 삼성그

In [18]:
for i, c in enumerate(result[:5000]):
  if c==59:

    print('idx:', i,'  -> ', news_contexts[i])
    print('length :',len(tokenizer.tokenize(news_contexts[i])))
    print()

idx: 535   ->  상장 임박 전망에 비트코인 가격 연초 대비 로 뛰어블랙록 등 대형사 참여로 기대감 고조 안정성 검증 등은 과제 비트코인 급등세 최재구 기자 서울 강남구 빗썸 고객센터에서 직원이 비트코인 차트를 살펴보고 있다. 세계 최대 자산운용사 블랙록의 비트코인 상장지수펀드 상장 신청 등의 영향으로 비트코인이 급등세를 보이고 있다. 한지훈 민선희 기자 비트코인 현물 상장지수펀드 가 내년 초 출시될 것으로 전망되면서 비트코인 가격 상승 기대감도 커지고 있다. 이 ETF가 증시에서 거래되면 대형 기관 투자자들은 그동안 각종 규제로 살 수 없었던 비트코인을 ETF 매수 방식으로 포트폴리오에 추가할 수 있게 된다. 기관들이 사실상 가상자산 투자 시장에 진출하게 되는 셈이다. 최근 가상자산 가격과 거래량이 지지부진한 흐름을 보이면서 크립토 윈터 라는 말까지 나오던 국면에 전환점이 될지 주목된다. 비트코인 비트코인 선물 이어 현물 ETF 승인되나 ETF는 인덱스펀드를 거래소에 상장해 주식처럼 사고팔 수 있도록 만든 상품이다. 개별 주식 등을 선택하지 않아도 되고, 상시 매매가 가능한 장점이 있다. 주식뿐 아니라 채권, 원자재 등을 ETF 형태 상품으로 거래한 지는 오래됐다. 가상자산 시장 관련 ETF도 이미 등장했다. 비트코인 선물을 기반으로 한 ETF는 이미 지난 부터 미국을 비롯한 글로벌 증시에 상장됐다. 미래의 비트코인 가격을 예측해 선물 계약을 매수하는 상품이었다. 대표적인 비트코인 선물 ETF인 BITO 가 뉴욕증권거래소에 상장됐으며, 레버리지 및 하락에 베팅하는 숏 상품들도 이미 상장목록에 올라 있다. 그러나 선물과 현물은 성격이 다르다는 이유로 비트코인 현물 ETF 상장 시도는 번번이 좌절됐다. 캐나다에서 세계 최초 비트코인 현물 ETF인 BTCC 가 상장됐지만, 금융 중심지인 미국에서는 아직 현물 ETF는 승인되지 않았다. 비트코인 선물 ETF인 BITO의 경우 미국 시카고거래소 의 비트코인 선물계약을 구입해 보유하는 구조다. 반면 비트코인 현